In [1]:
import numpy as np

def get_calib_from_file(calib_file):
    with open(calib_file) as f:
        lines = f.readlines()
    #修改3 改转换矩阵
    obj = lines[0].strip().split(' ')[1:]
    P2 = np.array(obj, dtype=np.float32)
    obj = lines[1].strip().split(' ')[1:]
    R0 = np.array(obj, dtype=np.float32)
    obj = lines[2].strip().split(' ')[1:]
    Tr_velo_to_cam = np.array(obj, dtype=np.float32)
    obj = lines[3].strip().split(' ')[1:]
    Tr_arbe_to_velo = np.array(obj, dtype=np.float32)
    obj = lines[4].strip().split(' ')[1:]
    Tr_ars_to_velo = np.array(obj, dtype=np.float32)
    
    obj = lines[5].strip().split(' ')[1:]
    Tr_arbe_to_cam = np.array(obj, dtype=np.float32)
    
    obj = lines[6].strip().split(' ')[1:]
    Tr_ars_to_cam = np.array(obj, dtype=np.float32)

    return {'P2': P2.reshape(3, 4),
            'Tr_arbe2velo': Tr_arbe_to_velo.reshape(3, 4),
            'Tr_ars2velo': Tr_ars_to_velo.reshape(3, 4),
            'R0': R0.reshape(3, 3),
            'Tr_velo2cam': Tr_velo_to_cam.reshape(3, 4),
            'Tr_arbe_to_cam': Tr_arbe_to_cam.reshape(3, 4),
            'Tr_ars_to_cam': Tr_ars_to_cam.reshape(3, 4)}

In [13]:
calib_ = get_calib_from_file('/root/code/OpenPCDet/data/dual_radar/radar_arbe/training/calib/000000.txt')
# print(calib_)

P2 = calib_['P2']  # 3 x 4
R0 = calib_['R0']  # 3 x 3
V2C = calib_['Tr_velo2cam']  # 3 x 4
#修改4 增加毫米波的转换矩阵
ARBE2V = calib_['Tr_arbe2velo'] # 3 x4
ARS2V = calib_['Tr_ars2velo'] # 3 * 4_
ARBE2C = calib_['Tr_arbe_to_cam'] # 3 x4
ARS2C = calib_['Tr_ars_to_cam'] # 3 * 4_

sensor2camera_r = ARBE2C[:3,:3]
sensor2camera_t = ARBE2C[:3, 3:4]
camera2sensor = np.eye(4).astype(np.float32)
camera2sensor[:3, :3] = sensor2camera_r.T
camera2sensor[3, :3] = -sensor2camera_t.reshape([1,3])
print(camera2sensor.T)

P2 = np.concatenate([P2, np.array([[0., 0., 0., 1.]])], axis=0)
print(P2)


arbe2image = P2 @ ARBE2C.T
print(arbe2image)
print(arbe2image.reshape([1,3,3]))

[[ 0.9987243   0.05038508  0.00333414 -0.20633236]
 [ 0.00245314  0.01753711 -0.9998432  -1.5064962 ]
 [-0.05043566  0.99857587  0.01739114 -1.9114609 ]
 [ 0.          0.          0.          1.        ]]
[[1.0642340e+03 0.0000000e+00 9.7494507e+02 0.0000000e+00]
 [0.0000000e+00 1.0657657e+03 6.1047357e+02 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00]]
[[ 1.0661270e+03 -9.7218146e+02 -3.6719940e+01]
 [ 5.5734100e+01 -5.9168738e+02  1.0748647e+03]
 [ 3.3341418e-03 -9.9984318e-01  1.7391138e-02]]
[[[ 1.0661270e+03 -9.7218146e+02 -3.6719940e+01]
  [ 5.5734100e+01 -5.9168738e+02  1.0748647e+03]
  [ 3.3341418e-03 -9.9984318e-01  1.7391138e-02]]]
